In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import os
from os.path import join
import logging
import json

import pandas as pd
from gridemissions.load import BaData
from gridemissions.eia_api_v1 import BAs, KEYS, SRC
import gridemissions
import re
import numpy as np

In [ ]:
from gridemissions.viz import set_plots

COLORS, PAGE_WIDTH, ROW_HEIGHT = set_plots()

from gridemissions.viz.reports import cleaning_plot, summ_stats
from matplotlib.backends.backend_pdf import PdfPages

from gridemissions.papers.physics_informed import (
    figure1,
    figure2,
    figure3,
    figure4,
    get_changes,
)

In [ ]:
print(gridemissions.config["APEN_PATH"])
FIG_PATH = join(gridemissions.config["APEN_PATH"], "figures")
os.makedirs(FIG_PATH, exist_ok=True)

# Load data

In [ ]:
file_name = join(gridemissions.config["APEN_PATH"], "data", "EBA_%s.csv")

raw = BaData(fileNm=file_name % "raw")
basic = BaData(fileNm=file_name % "basic")
rolling = BaData(fileNm=file_name % "rolling")
opt = BaData(fileNm=file_name % "opt")
opt_no_src = BaData(fileNm=file_name % "opt_no_src")

In [ ]:
co2 = BaData(fileNm=file_name % "co2", variable="CO2")

# Demand, generation for electricity & Consumed emissions

In [ ]:
save_figs = True

In [ ]:
ba_data = basic


sel = ba_data.df.index.month.isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
target_year = 2020

df = ba_data.df.loc[sel, :]
df = df.groupby([df.index.year, df.index.weekofyear]).median()

# Get rid of data on weeks 52/5a3 that have to do with timezoning
df = df.loc[df.index.get_level_values(1) < 40]

test = BaData(df=df)
data = BaData(df=df.loc[target_year, :] / 1e3)

# Run checkBA here as sanity
# for ba in test.regions:
#     test.checkBA("ba)

ref_years = [y for y in df.index.get_level_values(0).unique() if y < target_year]
ref_values = df.loc[ref_years, :].groupby(level=1).mean()
percent_changes = (df.loc[target_year, :] - ref_values) / ref_values * 100

percent_changes.head()


def make_plot(ba, save_figs=False):
    f, ax = plt.subplots(2, 3, figsize=(12, 4))
    ax = ax.flatten()
    cols = sum((ba_data.get_cols(ba, field) for field in ["D", "NG", "TI"]), [])
    titles = ["Demand", "Generation", "Net Interchange"]

    for year in df.index.get_level_values(0).unique():
        for i in range(len(cols)):
            if year == 2020:
                marker = "o"
                ms = 1
                ls = "-"
                lw = 2
            else:
                ls = "--"
                lw = 1.0
                marker = None
            ax[i].plot(
                df.loc[year, cols[i]] / 1e3, label=year, ls=ls, lw=lw, marker=marker
            )
            ax[i].set_title(titles[i])
            ax[i + 3].plot(percent_changes[cols[i]], color="k")

    for a in ax:
        a.grid()

    ax[0].set_ylabel("GW")
    ax[0].set_ylim(bottom=0)
    ax[1].set_ylim(bottom=0)
    ax[3].set_ylabel("Change from\n2016-2019 (%)")
    for a in ax[3:]:
        a.set_xlabel("Week of year")

    ax[2].legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    f.suptitle(ba, x=0.03)
    f.tight_layout()
    if save_figs:
        f.savefig(join(fig_folder, "%s.png" % ba))
        plt.close(f)


region_list = ["MISO", "PJM", "CISO", "BPAT", "SWPP", "ERCO", "PACE", "NYIS", "ISNE"]
for ba in region_list:
    make_plot(ba, save_figs=False)

In [ ]:
opt_no_src.df.columns

In [ ]:
df_merged = opt.df.copy(deep=True)
df_merged = df_merged.reindex(opt_no_src.df.index.union(opt.df.index))
for col in opt_no_src.df.columns:
    df_merged.loc[opt_no_src.df.index, col] = opt_no_src.df.loc[
        opt_no_src.df.index, col
    ]

In [ ]:
ba_dict = {
    "ERCO": "Electric Reliability Council of Texas",
    "MISO": "Midwest System Operator",
    "NYIS": "New York System Operator",
}

In [ ]:
prepend = ["(a) ", "(b) ", "(c) "]
for iba, ba in enumerate(["NYIS", "ERCO", "MISO"]):

    window_sz = 14

    ba_data = opt

    df_elec = (
        df_merged.loc[:, [c for c in df_merged.columns if ba in c]]
        .groupby(df_merged.index.date)
        .sum()
        .rolling(window_sz)
        .mean()
    )
    df_elec.index = pd.to_datetime(df_elec.index)
    df_elec.loc[:"20180701", [c for c in df_elec.columns if "SRC_" in c]] = np.nan

    df_co2 = (
        co2.df.loc[:, [c for c in co2.df.columns if ba in c]]
        .groupby(co2.df.index.date)
        .sum()
        .rolling(window_sz)
        .mean()
    )
    df_co2.index = pd.to_datetime(df_co2.index)

    scaling_elec = 1e-6
    scaling_co2 = 1e-9

    def make_plot(ax, s, color, label, window_sz=7, alpha=1.0, lw=1.0):
        xvals = pd.to_datetime(
            (10000 * 2020 + 100 * s.index.month + s.index.day).astype(str)
        )
        ax.plot(xvals, s, label=label, color=color, alpha=alpha, lw=lw)

    f, (ax, ax2) = plt.subplots(
        2,
        3,
        figsize=(1.5 * PAGE_WIDTH, 1.2 * ROW_HEIGHT),
        gridspec_kw={"height_ratios": [2.7, 1]},
    )
    for iyear, year in enumerate([2015, 2016, 2017, 2018, 2019, 2020]):
        if year == 2020:
            color = "k"
            alpha = 1
            lw = 1
        else:
            color = COLORS[iyear]
            alpha = 0.6
            lw = 0.5
        for ifield, field in enumerate(["D", "NG"]):
            make_plot(
                ax[ifield],
                df_elec.loc[
                    df_elec.index.year == year, ba_data.get_cols(r=ba, field=field)
                ]
                * scaling_elec,
                color=color,
                label=year,
                alpha=alpha,
                lw=lw,
                window_sz=21,
            )
        make_plot(
            ax[2],
            df_co2.loc[df_co2.index.year == year, co2.get_cols(r=ba, field="D")]
            * scaling_co2,
            color=color,
            label=year,
            alpha=alpha,
            lw=lw,
            window_sz=21,
        )

    import matplotlib.dates as mdates

    for a in ax:
        a.set_ylim(bottom=0)
        a.set_xticklabels([])
        a.tick_params(length=0.0)
        a.set_ylabel("TWh")
    ax[0].legend(loc=3, ncol=2)
    ax[0].set_title("Demand")
    ax[1].set_title("Generation")
    ax[2].set_title("Embodied emissions")
    ax[2].set_ylabel("Mton")

    percent_changes_elec = get_changes(df_elec)
    percent_changes_co2 = get_changes(df_co2)

    for ifield, field in enumerate(["D", "NG"]):
        ax2[ifield].plot(
            percent_changes_elec.loc[:, ba_data.get_cols(r=ba, field=field)]
        )
    ax2[2].plot(percent_changes_co2.loc[:, co2.get_cols(r=ba, field="D")])
    for a in ax2:
        a.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
        for label in a.get_xticklabels():
            label.set_ha("right")
            label.set_rotation(20)
        a.set_ylabel("%")
    f.suptitle(prepend[iba] + ba_dict[ba], ha="left", x=0.01, y=0.99)
    f.subplots_adjust(
        left=0.06, bottom=0.2, right=0.99, top=0.84, wspace=0.3, hspace=0.08
    )
    if save_figs:
        f.savefig(join(FIG_PATH, f"fig6_{ba}.pdf"))

In [ ]:
ba = "MISO"

window_sz = 14

ba_data = opt

src_cols = ["SRC_COL", "SRC_NG", "SRC_WND"]

df_elec = (
    df_merged.loc[:, [c for c in df_merged.columns if ba in c]]
    .groupby(df_merged.index.date)
    .sum()
    .rolling(window_sz)
    .mean()
)
df_elec.index = pd.to_datetime(df_elec.index)
for field in src_cols:
    df_elec.loc[:"20180710", ba_data.get_cols(r=ba, field=field)[0]] = np.nan


scaling_elec = 1e-6
scaling_co2 = 1e-9


def make_plot(ax, s, color, label, window_sz=7, alpha=1.0, lw=1.0):
    xvals = pd.to_datetime(
        (10000 * 2020 + 100 * s.index.month + s.index.day).astype(str)
    )
    ax.plot(xvals, s, label=label, color=color, alpha=alpha, lw=lw)


f, (ax, ax2) = plt.subplots(
    2,
    3,
    figsize=(1.5 * PAGE_WIDTH, 1.2 * ROW_HEIGHT),
    gridspec_kw={"height_ratios": [2.7, 1]},
)
for iyear, year in enumerate([2018, 2019, 2020]):
    if year == 2020:
        color = "k"
        alpha = 1
        lw = 1
    else:
        color = COLORS[iyear]
        alpha = 0.6
        lw = 0.5
    for ifield, field in enumerate(src_cols):
        make_plot(
            ax[ifield],
            df_elec.loc[df_elec.index.year == year, ba_data.get_cols(r=ba, field=field)]
            * scaling_elec,
            color=color,
            label=year,
            alpha=alpha,
            lw=lw,
            window_sz=21,
        )

import matplotlib.dates as mdates

for a in ax:
    a.set_ylim(bottom=0)
    a.set_xticklabels([])
    a.tick_params(length=0.0)
    a.set_ylabel("TWh")
ax[1].legend(loc=3, ncol=2)
ax[0].set_title("Coal")
ax[1].set_title("Gas")
ax[2].set_title("Wind")


def get_percent_changes(df, ref_years=[2018, 2019]):
    ref_data = df[df.index.year.isin(ref_years)]
    ref_values = ref_data.groupby([ref_data.index.month, ref_data.index.day]).mean()
    target_data = df[df.index.year.isin([2020])]
    target_values = target_data.groupby(
        [target_data.index.month, target_data.index.day]
    ).mean()

    percent_changes = (target_values - ref_values) / ref_values * 100
    percent_changes.index = pd.to_datetime(
        (
            10000 * 2020
            + 100 * percent_changes.index.get_level_values(0)
            + percent_changes.index.get_level_values(1)
        ).astype(str)
    )
    return percent_changes


percent_changes_elec = get_percent_changes(df_elec)
percent_changes_co2 = get_percent_changes(df_co2)

for ifield, field in enumerate(src_cols):
    ax2[ifield].plot(percent_changes_elec.loc[:, ba_data.get_cols(r=ba, field=field)])
# ax2[2].plot(percent_changes_co2.loc[:, co2.get_cols(r=ba, field="D")])
for a in ax2:
    a.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    for label in a.get_xticklabels():
        label.set_ha("right")
        label.set_rotation(30)
    a.set_ylabel("%")
f.subplots_adjust(left=0.07, bottom=0.2, right=0.99, top=None, wspace=0.3, hspace=0.05)
f.suptitle(ba_dict[ba])
f.suptitle(ba_dict[ba], ha="left", x=0.01, y=0.99)
f.subplots_adjust(left=0.06, bottom=0.2, right=0.99, top=0.84, wspace=0.3, hspace=0.08)
f.savefig(join(FIG_PATH, "fig8.pdf"))